# 数据清理

## URL 合并

In [2]:
# 导入数据文件

# here put the import lib
import os

import logging as log
import pandas as pd

from modules.logger.logger import init as logInit
from modules.logUtils.httpEvent import HttpEvent, TABLE_LABELS
from modules.fileUtils.fileUtils import getFilesByPath
from modules.config.argumentParser import argumentParser
from modules.config.configParser import getValue as getValueFromConfig

# CONFIG

TABLE_LABELS = [
    "timestamp",
    "threadId",
    "institutionId",
    "userId",
    "url",
    "method",
    "statusCode",
    "parameterType",
    "parameterName",
    "parameterValue",
    "header",
    "name",
    "ip",
    "port",
    "vpnip"
]
# get all files.
files = getFilesByPath("data/origin")
if not files: 
    print("there no any file. recorrect the input path.")
files = files

import random

# 随机抽取 RANDOM_FILES_NUMBER 数量的日志
RANDOM_FILES_NUMBER = 0
files = random.sample(files, RANDOM_FILES_NUMBER) if RANDOM_FILES_NUMBER != 0 else files

print(len(files))
print(files)

991
['data/origin/310111/fa/icaslog.log_2020-08-13.log', 'data/origin/310111/fa/icaslog.log_2020-08-30.log', 'data/origin/310111/fa/icaslog.log_2020-08-02.log', 'data/origin/310111/fa/icaslog.log_2020-08-07.log', 'data/origin/310111/fa/icaslog.log_2020-08-31.log', 'data/origin/310111/fa/icaslog.log_2020-08-09.log', 'data/origin/310111/fa/icaslog.log_2020-08-20.log', 'data/origin/310111/fa/icaslog.log_2020-08-22.log', 'data/origin/310111/fa/icaslog.log_2020-08-08.log', 'data/origin/310111/fa/icaslog.log_2020-08-19.log', 'data/origin/310111/fa/icaslog.log_2020-08-18.log', 'data/origin/310111/fa/icaslog.log_2020-08-21.log', 'data/origin/310111/fa/icaslog.log_2020-08-04.log', 'data/origin/310111/fa/icaslog.log_2020-08-29.log', 'data/origin/310111/fa/icaslog.log_2020-08-01.log', 'data/origin/310111/fa/icaslog.log_2020-08-12.log', 'data/origin/310111/fa/icaslog.log_2020-08-17.log', 'data/origin/310111/fa/icaslog.log_2020-08-11.log', 'data/origin/310111/fa/icaslog.log_2020-08-03.log', 'data/o

In [3]:
# 将数据读取到 DataFrame 中

data = []
for file in files:
    count = 0
    with open(file, 'r', encoding='utf-8') as f:
        # read raw log by line.
        for line in f:
            try:
                he = HttpEvent(line)
                
                # filter user
                if he.userId == 'null': continue

                # append to Data
                data.extend([he.getSet()])
                count += 1
            except Exception as e:
                log.debug(e)
    log.info("{0} write {1} lines".format(file, count))
print(len(data), len(data[0]))
# convert List to DataFrame
# print(data[0])
df = pd.DataFrame(data = data, columns=TABLE_LABELS)
del data

2424459 15


In [4]:
# 保存到 CSV
df.to_csv("data/tmp/full.csv", sep='^')

In [1]:
# 从 CSV 读取
import pandas as pd
df = pd.read_csv('data/tmp/full.csv', sep='^')

/opt/conda/envs/persona/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# 查看机构号
institutesDF = df.institutionId
institutes = institutesDF.value_counts()
for k, v in institutes.items():
    print(v, k)

1391274 241711
723433 310111
309752 321121


In [6]:
# 查看 userId
userIdDF = df.userId
users = userIdDF.value_counts()
for k, v in users.items():
    print(v, k)
print(len(users))

567775 241711-1053
92855 241711-762
79335 321121-680
44234 321121-237
26315 241711-1001060
24281 149492352b734619a613f2b1b593cdfb
24053 241711-3905
22869 310111-2217
21874 321121-986
20693 b2cb40368ffd4c568d28e5daf0341d36
20081 241711-8021
19372 321121-229
18605 7ba443d33df946caa96293206d962fb7
17911 241711-594
16909 241711-2710
15467 77e2aa653da8447888c024d2596cd29d
15460 241711-1000760
15338 310111-7461
15109 241711-1039
15027 321121-1006
14207 310111-7144
13760 241711-8698
13206 c7e364e631c94e6c89fe7480f7979ae6
12437 310111-6221
12334 310111-2703
11357 241711-8575
11234 241711-1001401
10959 241711-747
10777 241711-773
10695 241711-5982
10442 241711-7895
10406 321121-10000000
10113 241711-2482
10031 310111-2175
9697 241711-967
9694 321121-644
9618 241711-592
9430 321121-684
9383 310111-7179
9134 321121-891
9086 241711-1000940
8983 241711-1000641
8970 241711-4514
8950 310111-6834
8718 c1af98c1c20145a298cfcb30dbb49657
8578 321121-989
8520 241711-3969
8205 310111-2216
7980 66663321ec914

In [7]:
# 输出所有 url 和次数
urlDF = df.url
urls = urlDF.value_counts()

# for k, v in urls.items():
#     print(v, k)

print(len(urls))

27004


In [101]:
# def urlConvert(url: str)->str:
#     # filter url
#     subs = url.split('/')
#     for i, sub in enumerate(subs):
#         if sub == "" or i == 0: continue
#         if sub in userIdSet: subs[i] = "userId"
#         elif sub.split("-")[0] in instituteIdSet: subs[i] = "userId"
#         elif sub in instituteIdSet: subs[i] = "institutionId"
#         elif sub == he.name: subs[i] = "name"
#         elif len(sub) == 32: subs[i] = "hash"
#         elif 'By' in subs[i-1]:
#             subs[i] = subs[i-1].split('By')[-1]
#         elif 'code' in subs[i-1] or 'Code' in subs[i-1]:
#             subs[i] = 'code'
#         elif 'Id' in subs[i-1]:
#             subs[i] = 'id'
#         elif subs[i-1] == 'assetTag':
#             for j in range(i, len(subs)):
#                 subs[j] = "tag"
#             break
#         elif subs[i-1] == "queryMidAllActivities":
#             subs[i] = "activity"
#         elif "manualActivity" in sub:
#             subs[i] = "manualActivity"

#         elif subs[i-1] == "arBillType":
#             subs[i] = "type"
#         elif subs[i-1] == "isAlreadyEndAcpt" or subs[i-1] == "getPrjPartnerForDetail":
#             subs[i] = "userId"

#         elif subs[i-1] == "queryThroughLastActor":
#             subs[i] = "actor"
#         elif subs[i-1] == "getPrjBasicData":
#             subs[i] = "data"
#         elif subs[i-1] == "getSubPrjPage":
#             subs[i] = "page"
#         elif subs[i-1] == "changeExeBudgetUseState":
#             subs[i] = "state"
#         elif subs[i-1] == "updateTHOrgUnit" or subs[i-1] == "updateTHrAssignmentsNew":
#             for j in range(i, len(subs)):
#                 subs[j] = "unit"
#             break

#         elif subs[i-1].startswith("get"):
#             subs[i] = subs[i-1].split('get')[-1]
            
#         elif subs[i-1] == "role":
#             subs[i] = "role"
#         elif subs[i-1] == "tOdOfficeTemplates":
#             subs[i] = "id"
#         elif subs[i-1] == "updateTHrPeople":
#             subs[i] = "people"
#     return "/".join(subs)


# urlSet = set()

# with open("data/tmp/urlsConvert.csv", "w", encoding="utf-8") as target:
#     target.write("origin, convert\n")
#     for url in urlsSort.sort_values():
#         convertedUrl = urlFilter(url)
#         if url == convertedUrl and urls[url] < 500:
#             print("{0},{1}, {2}\n".format(urlFilter(url), url, urls[url]))
#             target.write("{0},{1}\n".format(urlFilter(url), url))


In [12]:
from modules.logUtils.urlFilter import urlConvert
# convert all urls
df['url'] = df['url'].apply(urlConvert, args=(set(userIdDF.values), set(institutesDF.values),  set(df.name.values),))

In [13]:
# 输出所有 url 和次数
urlDF = df.url
urls = urlDF.value_counts()

for k, v in urls.items():
    print(v, k)

print(len(urls))

279616 /fin/arBillTravel/findGwHomeTicketDataById
226194 /od/workflow/getFlowStepInfo
209892 /fin/arBill/findGwHomeReimConfig
189640 /fin/arBillTravel/getAllHoliday
84396 /od/workflow/getToDoList
61201 /portal/dictionary/codes/code/code
54231 /fin/arBillAssembly/getRefInfoList
50925 /portal/menus
41011 /hr/thrpeople/
38996 /fin/arRulesSetCommon/get/hash
34951 /portal/shortcutconfig/getConfigCompletion
30269 /od/tOdInHandle/submit
29751 /od/tOdArchivesInOrgids/hash
28179 /fin/arBill/getArBillAllTODOCount
28059 /pm/commoncore/todoCommonCore/getPmTotalPendingCount
26473 /portal/menus/url
23631 /od/tOdArchivesInOrgids
21926 /fin/arBillAssembly/getBankInfo/userId/id
21765 /fin/arExpenseLinkBillType/getExpenseLinkBillTypeSetByBillType/BillType
21680 /od/workflow/getAllTodoNumber
21616 /fa/commoncore/todoCommonCore/remoteGetTotalTodoNum
20832 /fin/arBill/getArBillAll
18944 /od/tOdQpjOrgids/hash
18719 /fin/arBillWF/doWorkFlow
16984 /portal/userusages
16554 /od/tOdArchivesOutOrgids/hash
15818 /

In [14]:
# 保存到 CSV
df.to_csv("data/tmp/converedtUrl_full.csv", sep='^')

In [21]:
class TemplateUrl(object):
    def __init__(self, s):
        self.template = s
        self.memIdx = []
# num > THRESHOULD，这就是一个固定模板 


        
def getTemplate(url: str)->[str]:
    subUrl = url.split('/')
    templates = []
    for i in range(1, len(subUrl)):
        pass

for url, urlNum in urls.items():
    if urlNum > THRESHOULD:
        print("yes", url)
    else:
        ts = getTemplate(url)
        print(ts)
    break

yes /fin/arBillMessage/send


In [18]:
# 按照userId进行主键排序，时间作为副键排序
users = df.sort_values(by=['userId', 'timestamp'])
users.to_csv('data/tmp/order.csv')

In [22]:
ii = df.groupby(by=['userId'])

In [85]:
def convertUrl(url: str)->str:
    subs = url.split('/')
    subs = subs[1:] if subs[0] == "" else subs
    # print(subs)
    for i, sub in enumerate(subs):
        if len(sub) == 32:
            subs[i] = "id"
        elif i > 1:
            if "By" in sub:
                subs[i] = sub[i-1].split('By')[-1]
            
            # if "ByCode" in subs[i-1] or "ByCodes" in subs[i-1] or "InCode" in subs[i-1]:
            #     subs[i] = "code"
            # elif "ById" in subs[i-1] or "ByIds" in subs[i-1]:
            #     subs[i] = "id"
    
    return "/" + "/".join(subs)

dic = dict()
for url in urls:
    template = convertUrl(url)
    dic[template] = 1 if template not in dic else dic[template] + 1

for k,v in dic.items():
    print(v, k)

1 /fa/asset/assetInfo/t/Id
1 /fa/commoncore/userInfo/remoteGetCurrenUserLeadrName/321121-120
1 /fa/commoncore/uniqueValidate/uniqueValidate/AssetInfo/assetTag/tags
1 /fa/ledger/accbook/updateCurrPeriod/hash
1 /fa/purchase/assetAcpt
1 /fa/ledger/inAssetDepreciate/getInassetDepreciateResult
1 /fa/ledger/voucherInfo/remoteGetPagedVoucherInfoQueryList
1 /fa/asset/assetModify/batchUpdateImportBizData
1 /fa/ledger/accAssetVoucherDepr
1 /fa/config/attachment/t
1 /fa/commoncore/assetCommonCore/getAssetClasss
1 /fa/inasset/inAssetModify/batchImportBizData
1 /fa/commoncore/bizWfMapping/t
1 /fa/ledger/assetDepreciateRecord/assetDepreciateRecord
1 /fa/asset/assetReceive/getPagedMyAssetApply
1 /fa/ledger/accbookPermission
1 /fa/ledger/period/remoteGetAllPeriod
1 /fa/config/externalConfig/e/Code
1 /fa/commoncore/prjInfo/getOpenedPrjInfoList
1 /fa/ledger/inAssetDepreciate
1 /fa/ledger/accbook/t
1 /fa/commoncore/assetCommonCore/remoteGetCurrenUserInfo
1 /fa/commoncore/wordFlow/remoteGetWorkFlowIsEnd
1